<a href="https://colab.research.google.com/github/cloudpendyala/pendyala_ai/blob/main/SentenceLevelChunking_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
!pip install PyMuPDF sentence-transformers faiss-cpu transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 54.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_

In [3]:
import fitz  # PyMuPDF

def chunk_pdf_paragraphs(pdf_path):
    document = fitz.open(pdf_path)
    paragraphs = []

    for page_num in range(len(document)):
        page = document[page_num]
        text = page.get_text("text")
        paragraphs.extend(text.split('\n\n'))  # Splitting by double newline for paragraphs

    # Clean empty paragraphs
    paragraphs = [p.strip() for p in paragraphs if p.strip()]
    return paragraphs

pdf_path = "/content/drive/My Drive/Colab/presidents.pdf"  # Change to your PDF path

paragraphs = chunk_pdf_paragraphs(pdf_path)



In [5]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("valhalla/t5-small-qg-hl")
model = T5ForConditionalGeneration.from_pretrained("valhalla/t5-small-qg-hl")

def generate_qa(paragraphs):
    qa_pairs = []
    for paragraph in paragraphs:
        input_text = "generate questions: " + paragraph
        input_ids = tokenizer.encode(input_text, return_tensors='pt')
        output_ids = model.generate(input_ids, max_length=512, num_beams=4, early_stopping=True)
        questions = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # Assuming the model returns questions separated by '<sep>'
        questions = questions.split('<sep>')

        # Generate answers using a simple approach for now
        for question in questions:
            if question.strip():
                qa_pairs.append((paragraph, question.strip(), paragraph))  # Using the paragraph as the answer
    return qa_pairs

qa_pairs = generate_qa(paragraphs)


tokenizer_config.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (677 > 512). Running this sequence through the model will result in indexing errors


In [6]:
print(qa_pairs)

[('President of the United\nStates\nPresidential seal\nPresidential flag\nIncumbent\nJoe Biden\nsince\xa0January 20, 2021\nExecutive branch of the U.S.\ngovernment\nExecutive Office of the President\nStyle\nMr. President[1][2]\n(informal)\nThe Honorable[3]\n(formal)\nHis Excellency[4][5]\n(diplomatic)\nType\nHead of state\nPresident of the United States\nThe president of the United States (POTUS)[B] is the head of state and\nhead of government of the United States of America. The president directs the\nexecutive branch of the federal government and is the commander-in-chief of\nthe United States Armed Forces.\nThe power of the presidency has grown substantially[12] since the first\npresident, George Washington, took office in 1789.[6] While presidential power\nhas ebbed and flowed over time, the presidency has played an increasingly\nsignificant role in American political life since the beginning of the 20th\ncentury, carrying over into the 21st century with notable expansions during t

In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')  # Using a more robust model

def generate_embeddings(texts):
    embeddings = model.encode(texts)
    return embeddings

paragraphs = [qa[0] for qa in qa_pairs]
answers = [qa[2] for qa in qa_pairs]

paragraph_embeddings = generate_embeddings(paragraphs)
answer_embeddings = generate_embeddings(answers)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
import faiss
import numpy as np

def store_embeddings_faiss(paragraphs, questions, answers, paragraph_embeddings, answer_embeddings):
    dimension = paragraph_embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)  # Use inner product (cosine similarity)

    combined_embeddings = np.concatenate((paragraph_embeddings, answer_embeddings), axis=0)
    faiss.normalize_L2(combined_embeddings)
    index.add(combined_embeddings)

    combined_data = [{'paragraph': paragraphs[i], 'question': questions[i], 'answer': answers[i]} for i in range(len(paragraphs))]
    combined_data += [{'paragraph': '', 'question': '', 'answer': answers[i]} for i in range(len(answers))]

    return index, combined_data

questions = [qa[1] for qa in qa_pairs]
index, stored_data = store_embeddings_faiss(paragraphs, questions, answers, paragraph_embeddings, answer_embeddings)


In [24]:
def search_paragraphs(query, index, stored_data, k=5):
    query_embedding = model.encode([query])
    query_embedding = np.array(query_embedding).reshape(1, -1)
    faiss.normalize_L2(query_embedding)
    distances, indices = index.search(query_embedding, k)

    results = []
    for i in range(k):
        match_percentage = distances[0][i] * 100  # Convert cosine similarity to percentage
        if match_percentage >= 30:
            result = stored_data[indices[0][i]]
            result['match_percentage'] = match_percentage
            results.append(result)

    results = sorted(results, key=lambda x: x['match_percentage'], reverse=True)  # Ensure sorting
    return results

query = "nixon"
results = search_paragraphs(query, index, stored_data)

for i, result in enumerate(results):
    print(f"Result {i + 1}:")
    print(f"Paragraph: {result['paragraph']}")
    print(f"Question: {result['question']}")
    print(f"Answer: {result['answer']}")
    print(f"Match Percentage: {result['match_percentage']:.2f}%\n")


Result 1:
Paragraph: 
Question: 
Answer: From left to right: Presidents George
H. W. Bush, Barack Obama, George
W. Bush, Bill Clinton, and Jimmy
Carter in the Oval Office on January
7, 2009; Obama took office thirteen
days later.
Some former presidents have had significant careers after leaving office.
Prominent examples include William Howard Taft's tenure as chief justice of
the United States and Herbert Hoover's work on government reorganization
after World War II. Grover Cleveland, whose bid for reelection failed in 1888,
was elected president again four years later in 1892. Two former presidents
served in Congress after leaving the White House: John Quincy Adams was
elected to the House of Representatives, serving there for 17 years, and Andrew
Johnson returned to the Senate in 1875, though he died soon after. Some ex-
presidents were very active, especially in international affairs, most notably
Theodore Roosevelt;[178] Herbert Hoover;[179] Richard Nixon;[180] and Jimmy
Carter.[1